In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD

import tensorflow as tf
from keras.models import Model
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.model import Sequential
from deel.lip.activations import GroupSort, FullSort
from tensorflow.keras.constraints import max_norm
from Autoencoderclasses import Encoder, Decoder, DenseEncoder, DenseDecoder
import os
LD_LIBRARY_PATH = '/home/wulab/anaconda3/lib/'
print(os.path.exists(os.path.join(LD_LIBRARY_PATH, "libcudart.so.11.0")))


print(tf.config.list_physical_devices('GPU'))
np.random.seed(123)

2023-05-20 18:14:27.941107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 18:14:28.336034: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-20 18:14:28.398597: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-20 18:14:28.398677: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

True
[]


2023-05-20 18:14:32.628144: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-20 18:14:32.628444: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-20 18:14:32.628502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-20 18:14:32.628549: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-20 18:14:32.653779: W tensorflow/c

In [2]:
import json
with open('data.json') as json_file:
    data_dict = json.load(json_file)



In [3]:
print(list(data_dict.keys())[200])
print(len(list(data_dict.values())[200]))
print(len(list(data_dict.keys())))

5.5,-200.0,-110.0
401
9261


In [4]:
data_points = []
for simulation in data_dict.values():
    for i in range(0,len(simulation),10):
        data_points.append(simulation[i])

data_points = np.array(data_points)



X_train, X_test, y_train, y_test = train_test_split(data_points ,data_points, test_size=0.2, random_state=123)
scaler_X = preprocessing.StandardScaler().fit(X_train)
scaler_y = preprocessing.StandardScaler().fit(y_train)


X_train = scaler_X.transform(X_train) 
X_test = scaler_X.transform(X_test) 
y_train = scaler_y.transform(y_train) 
y_test= scaler_y.transform(y_test)

print("y_train shape is {}".format(y_train.shape))


with open('meanandvar.npy', 'wb') as f:
    np.save(f,np.array(scaler_X.mean_))
    np.save(f,np.sqrt(np.array(scaler_X.var_)))
    np.save(f,np.array(scaler_y.mean_))
    np.save(f,np.sqrt(np.array(scaler_y.var_)))

import json
from pickle import dump

dump(scaler_X, open('scaler_X.pkl', 'wb'))
dump(scaler_y, open('scaler_y.pkl', 'wb'))

print(scaler_X.mean_)
print(scaler_X.var_)
print(scaler_y.mean_)
print(scaler_y.var_)


y_train shape is (303760, 46)
[0.         0.41308978 0.69049989 0.93965885 1.1505734  1.31712663
 1.43593837 1.50574973 1.52625246 1.49784779 1.42129797 1.29812005
 1.13108851 0.92575573 0.72862593 0.68780734 0.7323762  0.95539395
 1.18262314 1.37097919 1.51493613 1.61185639 1.66050622 1.66043322
 1.61163235 1.5145456  1.37039514 1.18180629 0.95429469 0.73099307
 0.68635883 0.72723764 0.92451469 1.12997335 1.29709011 1.42032648
 1.49692297 1.52537487 1.50492873 1.43518891 1.31646646 1.15002058
 0.93922963 0.69020705 0.41294157 0.        ]
[0.         0.93224997 1.39255556 2.0748696  2.84138307 3.57725098
 4.20191154 4.6692704  4.95609518 5.05958569 4.99348908 4.79198926
 4.51502966 4.25786737 4.10292635 3.99186314 4.12831521 4.30229361
 4.63445058 5.03660833 5.41041648 5.69149017 5.84120756 5.84137811
 5.69190296 5.41077806 5.03645717 4.63325889 4.29972307 4.12460308
 3.98789303 4.09902926 4.25519608 4.51401839 4.79240024 4.99484376
 5.06141933 4.958051   4.67110911 4.20349065 3.578499

In [6]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)
noise_size = [1.0]
lr = 5e-4
epochs = 10000
batch_size = 256
unit_list = [300] # please use even numbers
reduced_dim_list =[6]
results_dict = dict({})
for noise in noise_size:
    noise_arr = np.random.normal(0,noise,size = [len(y_train),len(y_train[0])])
    X_train_noise = X_train + noise_arr 
    y_train_noise = y_train + noise_arr 
    for unit in unit_list:
        for rd in reduced_dim_list:

            spectral_model = tf.keras.Sequential()
            spectral_model.add(Encoder(units = unit, reduced_dim = 6 ))
            spectral_model.add(Decoder(units = unit, output_dim = 46 ))
            spectral_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mean_squared_error')
            spectral_model.fit(X_train_noise, y_train_noise, epochs=epochs , batch_size= batch_size , validation_split=0.25, verbose=2, callbacks=[callback])
            spectral_model.evaluate(X_test,y_test)
            
            title = "SpectralDense " + str(unit) + " units " + str(rd) + " rd" + " " + str(noise) + " noise.h5"
            results_dict[title] = spectral_model.evaluate(X_test,y_test)
            spectral_model.save(title)


            model = tf.keras.Sequential()
            model.add(tf.keras.Input(shape=(46,)))
            model.add(DenseEncoder(units = unit, reduced_dim = rd ))
            model.add(DenseDecoder(units = unit, output_dim = 46 ))
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mean_squared_error')
            model.fit(X_train_noise, y_train_noise, epochs=epochs , batch_size= batch_size , validation_split=0.25, verbose=2, callbacks=[callback])
            model.evaluate(X_test,y_test)
            model.save("Dense " + str(unit) + " units " + str(rd) + " rd" + " " + str(noise) + " noise.h5")
            title = "Dense " + str(unit) + " units " + str(rd) + " rd" + " " + str(noise) + " noise.h5"
            results_dict[title] = model.evaluate(X_test,y_test)


spectral_model.evaluate(X_test,y_test)
model.evaluate(X_test,y_test)



Epoch 1/10000
890/890 - 4s - loss: 0.9328 - val_loss: 0.8792 - 4s/epoch - 5ms/step
Epoch 2/10000
890/890 - 3s - loss: 0.8761 - val_loss: 0.8764 - 3s/epoch - 3ms/step
Epoch 3/10000
890/890 - 3s - loss: 0.8738 - val_loss: 0.8747 - 3s/epoch - 3ms/step
Epoch 4/10000
890/890 - 3s - loss: 0.8719 - val_loss: 0.8728 - 3s/epoch - 3ms/step
Epoch 5/10000
890/890 - 3s - loss: 0.8698 - val_loss: 0.8705 - 3s/epoch - 3ms/step
Epoch 6/10000
890/890 - 3s - loss: 0.8678 - val_loss: 0.8691 - 3s/epoch - 3ms/step
Epoch 7/10000
890/890 - 3s - loss: 0.8659 - val_loss: 0.8678 - 3s/epoch - 3ms/step
Epoch 8/10000
890/890 - 3s - loss: 0.8642 - val_loss: 0.8660 - 3s/epoch - 3ms/step
Epoch 9/10000
890/890 - 3s - loss: 0.8626 - val_loss: 0.8648 - 3s/epoch - 3ms/step
Epoch 10/10000
890/890 - 3s - loss: 0.8612 - val_loss: 0.8637 - 3s/epoch - 3ms/step
Epoch 11/10000
890/890 - 3s - loss: 0.8600 - val_loss: 0.8627 - 3s/epoch - 3ms/step
Epoch 12/10000
890/890 - 3s - loss: 0.8588 - val_loss: 0.8621 - 3s/epoch - 3ms/step
E

0.10767888277769089

: 

In [ ]:
f = open('results_dict')
data = json.load(f)

data.update(results_dict)

with open("results_dict", "w") as fp:
    json.dump(data,fp)

: 

[[0.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 0.0], [0.0, 3.5685354858030944, 6.030640248311903, 6.030640248311903, 6.030640248311903, 6.030640248311903, 6.030640248311903, 6.030640248311903, 6.030640248311903, 6.030640248311902, 6.030640248289456, 6.030640066423089, 6.030440783007881, 6.001036972410699, 5.436042557150956, 4.414356149185403, 5.43604255715096, 6.001036972410699, 6.030440783007881, 6.030640066423089, 6.030640248289456, 6.030640248311902, 6.030640248311903, 6.030640248311903, 6.030640248311902, 6.030640248289456, 6.030640066423089, 6.030440783007881, 6.001036972410698, 5.436042557150952, 4.414356149185403, 5.43604255715096, 6.001036972410699, 6.030440783007881, 6.030640066423089, 6.030640248289456, 6.030640248311902, 6.030640248311903, 6.030640248311903, 6.030640248311903, 6.030640248311903, 6.